In [2]:
import numpy as np
import scipy
from scipy.special import gamma
from scipy.special import beta
import scipy.integrate as integrate

In [14]:
def genParam1(n, randomstate=0) :
    r = np.random.RandomState(seed=randomstate)
    
    alpha = r.exponential(1,(n,3))
    ksi = np.abs(r.standard_cauchy(2))
    beta = [r.dirichlet(alpha[0])]
    for i in range(1,n) :
      betai = [r.dirichlet(alpha[i])]
      beta = np.concatenate((beta,betai), axis=0)
    
    mu = [[beta[0][0]-beta[0][1], 2*(beta[0][0]+beta[0][1])-1]]
    for i in range(1,n) :
      mui = [[beta[i][0]-beta[i][1], 2*(beta[i][0]+beta[i][1])-1]]
      mu = np.concatenate((mu, mui),axis=0)
    sigma = r.wald(ksi[0], ksi[1],n)
    
    return(alpha, ksi, beta, mu, sigma)

In [15]:
def genParam2(n, randomstate=0) :
    r = np.random
    
    alpha = r.exponential(1,(n,3))
    ksi = np.abs(r.standard_cauchy(2))
    beta = [r.dirichlet(alpha[0])]
    for i in range(1,n) :
      betai = [r.dirichlet(alpha[i])]
      beta = np.concatenate((beta,betai), axis=0)
    
    mu = [[beta[0][0]-beta[0][1], 2*(beta[0][0]+beta[0][1])-1]]
    for i in range(1,n) :
      mui = [[beta[i][0]-beta[i][1], 2*(beta[i][0]+beta[i][1])-1]]
      mu = np.concatenate((mu, mui),axis=0)
    sigma = r.wald(ksi[0], ksi[1],n)
    
    return(alpha, ksi, beta, mu, sigma)

In [19]:
def genX(n, T, mu, sigma) :
    A = np.eye(T, T+2) + mu[0][0]*np.eye(T, T+2,1) + mu[0][1]*np.eye(T, T+2,2)
    A = A@A.T
    X = [np.random.multivariate_normal(np.zeros(T), sigma[0]*A)]
    for i in range(1,n) :
      A = np.eye(T, T+2) + mu[i][0]*np.eye(T, T+2,1) + mu[i][1]*np.eye(T, T+2,2)
      A = sigma[i]*A@A.T
      Xi = [np.random.multivariate_normal(np.zeros(T), A)]
      X = np.concatenate((X,Xi), axis=0)
    return(X)

In [103]:
PROBA_MUMHprop1 = 1/2
PROBA_MUMHprop2 = 1/2
PROBA_MUMH1 = 1/2
PROBA_MUMH2 = 1/2

T = 100
X = genX(1,T, genParam1(1, 0)[3], genParam1(1, 0)[4])
epsilon = 1

muMH = [np.random.uniform(-1,1,2)]
sigmaMH = [np.abs(np.random.normal(0,1))]

i = 0

for j in range(1) :
    
    AMH = np.eye(T, T+2) + muMH[i][0]*np.eye(T, T+2,1) + muMH[i][1]*np.eye(T, T+2,2)
    AMH = sigmaMH[i]*(AMH@AMH.T)

    muMHprop = np.random.uniform(-1,1,2)
    sigmaMHprop = np.abs(np.random.normal(sigmaMH[i],1))

    AMHprop = np.eye(T, T+2) + muMHprop[0]*np.eye(T, T+2,1) + muMHprop[1]*np.eye(T, T+2,2)
    AMHprop = sigmaMHprop*(AMHprop@AMHprop.T)

    rnum = scipy.stats.multivariate_normal.pdf(X, np.zeros(T), AMHprop)*PROBA_MUMHprop1*PROBA_MUMHprop2
    rnum = rnum*scipy.stats.invgauss.pdf(sigmaMHprop, genParam1(1, 0)[1][0]/genParam1(1, 0)[1][1], 0, genParam1(1, 0)[1][1]) 
    rnum = rnum*scipy.stats.uniform.pdf(muMH[i][0],-1,2)*scipy.stats.uniform.pdf(muMH[i][1],-1,2)
    rnum = rnum*scipy.stats.foldnorm.pdf(sigmaMH[i], 0, sigmaMHprop, 1)
    
    rden = scipy.stats.multivariate_normal.pdf(X, np.zeros(T), AMH)*PROBA_MUMH1*PROBA_MUMH2
    rden = rden*scipy.stats.invgauss.pdf(sigmaMH[i], genParam1(1, 0)[1][0]/genParam1(1, 0)[1][1], 0, genParam1(1, 0)[1][1])    
    rden = rden*scipy.stats.uniform.pdf(muMHprop[0],-1,2)*scipy.stats.uniform.pdf(muMHprop[1],-1,2)
    rden = rnum*scipy.stats.foldnorm.pdf(sigmaMHprop, 0, sigmaMH[i], 1)
    
    if rden == 0 :
        r = 0
    else : 
        r = min(1, rnum/rden)
        
    if np.random.uniform()<= r :
        muMH = np.concatenate((muMH, [muMHprop]), axis=0)
        sigmaMH.append(sigmaMHprop)
    else :
        muMH = np.concatenate((muMH, [muMH[i]]), axis=0)
        sigmaMH.append(sigmaMH[i])

    i+=1


2.1661225642730578e-68
6.219017047768435e-69
1.5547542619421088e-69
0.0 0.0


In [110]:
scipy.stats.multivariate_normal.pdf(X, np.zeros(T), AMHprop)

8.664490257092231e-68

In [55]:
def probmu2(x, u, v) :
  if (x<-1) | (x>1) :
    return(0)
  else :
    return(2*(((x+1)/2)**(u-1))*((1-(x+1)/2)**(v-1))/beta(u,v))

In [ ]:
def probmu1(x, u, v, z) :
  if (z<0) | (z>1-x) :
    return(0)
  else :
    return( (integrate(lambda x : ))   )

In [ ]:
def f(X,theta) :
  psigma = (np.sqrt(theta[4]/(2*np.pi*theta[10]**3)))*np.exp((-theta[4]*(theta[10]-theta[3])**2)/(2*theta[10]*theta[3]**2))
  pksi1 = 2/(np.pi*(1+theta[3]**2))
  pksi2 = 2/(np.pi*(1+theta[4]**2))
  pmu1 =
  pmu2 = probmu2(theta[9],theta[0]+theta[1], theta[2])
  pbeta = (theta[5]**(theta[0]-1))*(theta[6]**(theta[1]-1))*(theta[7]**(theta[1]-1))*gamma(sum(theta[0:3])/(gamma(theta[0])*gamma(theta[1])*gamma(theta[2]))
  palpha = np.exp(-(sum(theta[:3])))

  return(psigma*pksi1*pksi2*pmu1*pmu2*pbeta*palpha)

In [43]:
T=100
A = np.eye(T, T+2) + genParam1(1, 0)[3][0][0]*np.eye(T, T+2,1) + genParam1(1, 0)[3][0][1]*np.eye(T, T+2,2)
A = genParam1(1, 0)[4][0]*(A@A.T)
scipy.stats.multivariate_normal.pdf(genX(1,T, genParam1(1, 0)[3], genParam1(1, 0)[4]), np.zeros(T), A)

6.276157733181793e-49